# 02 · Single GMRES run

Drive the GMRES solver for a single configuration and track residuals.

### Load the essentials

In [ ]:
# %% Setup (hot-reload while you tweak src/*)
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt

# Core API from your package
from src import GridSpec, GMRESOptions, gmres_solve

# Operators / loads / viz helpers
from src.operators import helmholtz_operator, BC
from src.loads import PointSource
from src.visualisation import plot_residuals, plot_field

## Versions

Useful when comparing runs across environments (Python / SciPy / package).


In [ ]:
try:
    import scipy
except Exception:
    scipy = None

try:
    import src as pkg
    pkg_version = getattr(pkg, "__version__", "unknown")
except Exception:
    pkg_version = "unknown"

print("Python :", sys.version.split()[0])
print("NumPy  :", np.__version__)
print("SciPy  :", getattr(scipy, "__version__", "not installed"))
print("Package:", pkg_version)


## Parameters

Tweak everything here: grid size, wavenumber `k`, boundary condition `bc`, dtype, and GMRES options.


In [ ]:
# Grid
shape   = (60, 60)
lengths = (1.0, 1.0)

# Helmholtz
k     = 30.0                 # wavenumber |k|
bc    = BC.DIRICHLET         # try: BC.NEUMANN, BC.PERIODIC
dtype = "complex128"         # "float64" if you want a purely real system

# Right-hand side (point source)
source_location = "centre"   # or (x, y) in physical coords, or "random"
source_amp      = 1.0

# GMRES options
gmres_opts = GMRESOptions(tol=1e-6, maxiter=200, restart=None)


## Build grid


In [ ]:
grid = GridSpec(dims=2, shape=shape, lengths=lengths)
grid


## Assemble Helmholtz operator

Build $A = \Delta + k^2 I$ with the selected boundary condition.


In [ ]:
A = helmholtz_operator(grid, k=k, bc=bc, dtype=dtype)
A.shape, A.nnz


## Build the RHS (point source)

`PointSource` understands keywords (`"centre"`, `"origin"`, `"random"`) or physical coordinates.


In [ ]:
load = PointSource(location=source_location, amplitude=source_amp)
b = load.build(grid)
b.shape, (b != 0).sum()    # show size and number of non-zeros (should be 1)


## Run GMRES

Solve \(A u = b\) and report convergence.  
Residual history is recorded through the options-powered callback in `gmres_solve`.


In [ ]:
importlib.reload(solvers)


In [ ]:
print("Using solvers module from:", solvers.__file__)


In [ ]:
print("Wrapper passes rtol? ->", "rtol" in inspect.getsource(solvers.gmres_solve))


In [ ]:
gmres_opts = GMRESOptions(tol=1e-6, maxiter=200, restart=None)


In [ ]:
result = gmres_solve(A, b, options=gmres_opts)


In [ ]:
# Run the solver
result = gmres_solve(A, b, options=gmres_opts)

# Print summary
print(f"Converged: {result.converged} | info: {result.info}")

if result.residuals:
    print(f"Iterations: {len(result.residuals)}")
    print(f"Final residual: {result.residuals[-1]:.2e}")


In [ ]:

# Run the solver
result = gmres_solve(A, b, options=gmres_opts)

# Print summary
print(f"Converged: {result.converged} | info: {result.info}")
if result.residuals:
    print(f"Iterations: {len(result.residuals)} | Final residual: {result.residuals[-1]:.2e}")



## Diagnostics

Plots for
- residual history,
- real part of the solution field,
- magnitude of the solution field. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.visualisation import plot_residuals, plot_field

#### Residual Convergence


In [ ]:
if hasattr(result, "residuals") and result.residuals:
    ax = plot_residuals(result)  # your built-in plotting helper
    ax.figure.suptitle("GMRES Residual Convergence")
    plt.show()
else:
    print("No residual history available to plot.")

#### Real Part of Solution Field

In [ ]:
if hasattr(result, "solution") and result.solution is not None:
    u = np.array(result.solution)
    u_real = np.real(u).reshape(grid.shape)
    ax = plot_field(u_real, grid.shape)
    ax.figure.suptitle("Real Part of Solution Field")
    plt.show()
else:
    print("No solution vector found in GMRES result.")

#### Magnitude field

In [ ]:
if hasattr(result, "solution") and result.solution is not None:
    u_abs = np.abs(result.solution).reshape(grid.shape)
    plt.figure()
    plt.imshow(u_abs, extent=[0, grid.lengths[0], 0, grid.lengths[1]], origin="lower")
    plt.colorbar(label="|u|")
    plt.title("Solution Magnitude Field")
    plt.xlabel("x"); plt.ylabel("y")
    plt.tight_layout()
    plt.show()

### Next steps

- Try `bc = BC.NEUMANN` or `BC.PERIODIC` and compare convergence.
- Sweep over `k` (e.g., 10, 20, 30, 40) to see how it affects GMRES.
- Replace `PointSource` with `PlaneWaveSource` or `RandomSource` for different RHS types.
- Switch `dtype` to `"float64"` if your loads/solutions are real-valued.
